## Test Script for Early Analysis

Dieses Skript dient dazu, sich mit der Datnstruktur von [oeffentlichevergabe.de](https://oeffentlichevergabe.de/ui/de/) vetraut zu machen.

Ziel des Skript ist es:
- Zu überprüfen wie vollständig die Daten sind
- Ob und welche Risikoindikatoren auf Grundlage der Daten gebildet werden können

## 1. Libraries

In [ ]:

import pandas as pd # for prduction change to polas?

#Improve paths later on. Run everything from project root without notebooks
import sys
import os
# Add project root (two levels up) to sys.path
sys.path.append(os.path.abspath(os.path.join('..', '..')))

# Import risk analysisi functions
from utils.analysis.risk_analysis import analyze_risk


In [ ]:
# Test risk analysis function
analyze_risk('test')

testing imports


## 2. Load & Examine Structure of individual xlsx sheets
note: The individual csv files that the main.py scropt doenloads are combined into one xlsx file (stand: 18.12.25). This provides the main entry point for us to investigate the retrieved data

In [17]:
# Load the Excel file
df1 = pd.ExcelFile('../../data/notices_2025-12-06.xlsx')

In [32]:
'''for sheet_name in df1.sheet_names:
    df = df1.parse(sheet_name)
    print(f"\n=== {sheet_name} ===")
    print(df.columns.tolist())
    print(df.head(2))  # optional but very helpful
'''

'for sheet_name in df1.sheet_names:\n    df = df1.parse(sheet_name)\n    print(f"\n=== {sheet_name} ===")\n    print(df.columns.tolist())\n    print(df.head(2))  # optional but very helpful\n'

In [7]:
# List all sheet names
for sheet_name in df.sheet_names:
    print(sheet_name)

additionalInformation
changes
classification
contract
cvdInformation
duration
lot
notice
noticeResult
organisation
placeOfPerformance
procedure
procedureLotResult
purpose
receivedSubmissions
secondStage
strategicProcurement
submissionTerms
tender


In [13]:
# testing organisation data
organisation = pd.read_excel('../../data/notices_2025-12-06.xlsx',sheet_name='organisation')
organisation.head(2)

,noticeIdentifier,noticeVersion,organisationName,organisationIdentifier,organisationCity,organisationPostCode,organisationCountrySubdivision,organisationCountryCode,organisationInternetAddress,organisationNaturalPerson,organisationRole,buyerProfileURL,buyerLegalType,buyerContractingEntity,winnerSize,winnerOwnerNationality,winnerListed
0,18d7ea5a-47a5-49e2-b7c0-676c517d32c6,1,Landkreis Cuxhaven,Leitweg-ID 03352-0-30,Cuxhaven,27474,DE932,DEU,https://www.landkreis-cuxhaven.de,NaN,buyer,NaN,koerp-oer-kommun,NaN,NaN,NaN,NaN
1,18d7ea5a-47a5-49e2-b7c0-676c517d32c6,1,Vergabekammer Niedersachsen beim Nds. Minister...,t:04131153308,Lüneburg,21339,DE935,DEU,NaN,NaN,reviewer,NaN,NaN,NaN,NaN,NaN,NaN


In [37]:
# testing tender data
tender = pd.read_excel('../../data/notices_2025-12-06.xlsx',sheet_name='tender')
tender.head(2)

,noticeIdentifier,noticeVersion,tenderIdentifier,lotIdentifier,tenderValue,tenderValueCurrency,tenderPaymentValue,tenderPaymentValueCurrency,tenderPenalties,tenderPenaltiesCurrency,tenderRank,concessionRevenueUser,concessionRevenueUserCurrency,concessionRevenueBuyer,concessionRevenueBuyerCurrency,countryOrigin
0,18d7ea5a-47a5-49e2-b7c0-676c517d32c6,1,TEN-0001,LOT-0001,571400.00,EUR,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,a74e1c14-073f-4e93-8ee1-93d97ebc47bd,1,TEN-0001,LOT-0001,800684.85,EUR,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## 3. Test: SINGLE BIDDING INDICATOR

In [ ]:
# get: procedureLotResult
procedureLotResult_df = pd.read_excel('../../data/notices_2025-12-06.xlsx',sheet_name='procedureLotResult')
procedureLotResult_df.head(1)

,noticeIdentifier,noticeVersion,lotIdentifier,procedureLotResultNumber,winnerChosen,notAwardedReason,frameworkMaximumValue,frameworkMaximumValueCurrency,frameworkEstimatedValue,frameworkEstimatedValueCurrency,tenderValueLowest,tenderValueLowestCurrency,tenderValueHighest,tenderValueHighestCurrency
0,18d7ea5a-47a5-49e2-b7c0-676c517d32c6,1,LOT-0001,0,selec-w,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [35]:
# get: receivedSubmissions 
receivedSubmissions_df = pd.read_excel('../../data/notices_2025-12-06.xlsx',sheet_name='receivedSubmissions')
receivedSubmissions_df.head(1)

,noticeIdentifier,noticeVersion,procedureLotResultNumber,receivedSubmissionsCount,receivedSubmissionsType
0,18d7ea5a-47a5-49e2-b7c0-676c517d32c6,1,0,3,t-esubm


In [ ]:
# merge both dataframes to get bidder count per lot
bidder_df = pd.merge(procedureLotResult_df, receivedSubmissions_df, on=['noticeIdentifier','noticeVersion','procedureLotResultNumber'], how='inner')

In [ ]:
# create single bidder indicator
bidder_df['single_bidder'] = bidder_df['receivedSubmissionsCount'] == 1

In [38]:
# 
bidder_df[bidder_df['single_bidder']==True].head(5)

,noticeIdentifier,noticeVersion,lotIdentifier,procedureLotResultNumber,winnerChosen,notAwardedReason,frameworkMaximumValue,frameworkMaximumValueCurrency,frameworkEstimatedValue,frameworkEstimatedValueCurrency,tenderValueLowest,tenderValueLowestCurrency,tenderValueHighest,tenderValueHighestCurrency,receivedSubmissionsCount,receivedSubmissionsType,single_bidder
7,fed757d9-5a74-4b74-a4bb-7aecd6ac2b33,1,LOT-0001,0,selec-w,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,t-esubm,True
8,f0ec246d-648e-4214-8863-615eb450a566,1,LOT-0001,0,selec-w,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,t-esubm,True
14,91921f44-d893-491b-9192-3b6d647d1c1d,1,LOT-0001,0,selec-w,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,t-esubm,True
15,796f7d05-9ca2-44a2-9ced-1b4a0780ed8f,1,LOT-0001,0,selec-w,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,t-esubm,True
16,82c99204-73fd-45b8-8867-d0d27c6a5a73,1,LOT-0001,0,selec-w,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,t-esubm,True
